In [ ]:
import serial as sr
import pandas as pd
import struct
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import cv2


In [ ]:
#ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)

def read_until_end(serial_port, end_marker="end"):
    received_data = bytearray()
    end_marker_bytes = end_marker.encode()

    while True:
        if serial_port.in_waiting > 0:
            data = serial_port.read(serial_port.in_waiting)
            received_data.extend(data)

            if end_marker_bytes in received_data:
                break

    # Remove the end marker from the received data
    received_data = received_data.split(end_marker_bytes)[0]
    return received_data

rows = 24
columns = 16
values = np.zeros(shape=(rows, columns), dtype='float32')
#print(values)

while 1:
    ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)
    ser.flush()
    ser.write("Ready\r".encode('UTF-8'))
    
    value = read_until_end(ser)
    #print(value)
    if len(value) == 3072:
        i = 0
        
        r = 0
        while r<24:
            c = 0
            k = 0
            while k<32:
                x = struct.unpack('<f', value[i:i+4])[0]
                if x != 0:
                    values[r,c] = x
                    c+=1
                k+=1
                i+=4                
            r+=1
        #print(values)
    fig = plt.figure()
    anim = animation.FuncAnimation(fig, sns.heatmap(values), init_func=sns.heatmap(values), frames=8, repeat = True)
    plt.show()
#    return_val = ser.readline()
    

In [ ]:
import sys
len(value)

In [ ]:


sns.heatmap(values)

In [ ]:
import numpy as np
def uint16_to_float(uint16_value):
    # Convert the uint16 value to bytes
    bytes_value = uint16_value.to_bytes(2, byteorder='little', signed=False)
    
    # Interpret the bytes as a half-precision float using numpy
    float16_value = np.frombuffer(bytes_value, dtype=np.float16)[0]
    
    return float16_value


np.array(byte_list, dtype=np.uint8)

ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)
i = 0
while i<2:
    value = ser.read(3072)
    #valueinstring = str(value, 'UTF-8')
    #print(value)
    byte_list.extend(value)
    i+=1
byte_list = byte_list.to_bytes(4, byteorder='little', signed=False)
np.frombuffer(bytes_list, dtype=np.float16)

In [ ]:
import serial as sr
import pandas as pd
import struct
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import cv2
import os


#ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)

def read_until_end(serial_port, end_marker="end"):
    received_data = bytearray()
    end_marker_bytes = end_marker.encode()

    while True:
        if serial_port.in_waiting > 0:
            data = serial_port.read(serial_port.in_waiting)
            received_data.extend(data)

            if end_marker_bytes in received_data:
                break

    # Remove the end marker from the received data
    received_data = received_data.split(end_marker_bytes)[0]
    return received_data

def save_heatmaps_as_images(heatmaps, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for i, heatmap in enumerate(heatmaps):
        plt.figure()
        sns.heatmap(heatmap)
        plt.savefig(f"{folder}/heatmap_{i}.png")
        plt.close()

# Create video from images
def create_video_from_images(folder, output_file, fps):
    images = [img for img in os.listdir(folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0])) # Sort images by frame number
    
    frame = cv2.imread(os.path.join(folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    
    for image in images:
        video.write(cv2.imread(os.path.join(folder, image)))
    
    cv2.destroyAllWindows()
    video.release()


rows = 24
columns = 16
values = np.zeros(shape=(rows, columns), dtype='float32')
#print(values)
frames = []
vid_size = 80
current_frame = 0

while current_frame<vid_size:
    ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)
    ser.flush()
    ser.write("Ready\r".encode('UTF-8'))
    
    value = read_until_end(ser)
    #print(value)
    if len(value) == 3072:
        i = 0
        
        r = 0
        while r<24:
            c = 0
            k = 0
            while k<32:
                x = struct.unpack('<f', value[i:i+4])[0]
                if x != 0:
                    values[r,c] = x
                    c+=1
                k+=1
                i+=4                
            r+=1
        #print(values)
        frames.append(values)
        current_frame+=1
    
#print(frames)
num_frames = 80
folder = 'heatmaps'
output_file = 'heatmap_video.avi'
fps = 8

save_heatmaps_as_images(frames, folder)
create_video_from_images(folder, output_file, fps)

#    return_val = ser.readline()
    

In [ ]:
import serial as sr
import pandas as pd
import struct
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import cv2
import os


def save_heatmaps_as_images(heatmaps, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for frame_number, heatmap in enumerate(heatmaps):
        
        if frame_number == 0:
            np_matrix = np.array(frames)
            max_val = np_matrix.max()
            min_val = np_matrix.min()
        
        plt.figure()
        sns.heatmap(heatmap, xticklabels = False, yticklabels = False)
        plt.savefig(f"{folder}/heatmap_{frame_number}.png")
        plt.close()

#ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)

def read_until_end(serial_port, end_marker="end"):
    received_data = bytearray()
    end_marker_bytes = end_marker.encode()

    while True:
        if serial_port.in_waiting > 0:
            data = serial_port.read(serial_port.in_waiting)
            received_data.extend(data)

            if end_marker_bytes in received_data:
                break

    # Remove the end marker from the received data
    received_data = received_data.split(end_marker_bytes)[0]
    return received_data

# Create video from images
def create_video_from_images(folder, output_file, fps):
    images = [img for img in os.listdir(folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0])) # Sort images by frame number
    #print(int(x.split('_')[1].split('.')[0]))
    frame = cv2.imread(os.path.join(folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    
    for image in images:
        video.write(cv2.imread(os.path.join(folder, image)))
    
    cv2.destroyAllWindows()
    video.release()


rows = 24
columns = 16

#print(values)

vid_size = 80
current_frame = 0
folder = 'heatmaps'
output_file = 'heatmap_video.avi'
fps = 8
frames = []

while current_frame<vid_size:
    ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)
    ser.flush()
    ser.write("Ready\r".encode('UTF-8'))
    
    value_from_sensor = read_until_end(ser)
    #print(value)
    values_in_array = np.zeros(shape=(rows, columns), dtype='float32')
    if len(value_from_sensor) == 3072:
        i = 0
        
        r = 0
        while r<24:
            c = 0
            k = 0
            while k<32:
                x = struct.unpack('<f', value_from_sensor[i:i+4])[0]
                # if x != 0:
                #     values_in_array[r,c] = x
                #     c+=1
                values_in_array[r,c] = x
                c+=1
                k+=1
                i+=4
            r+=1
        #print(values_in_array)
        print(current_frame)
        #print("     ")
        frames.append(values_in_array)    
        current_frame+=1
            
print("done Capturing")
save_heatmaps_as_images(frames, folder)
create_video_from_images(folder, output_file, fps)

#    return_val = ser.readline()
    

In [ ]:
folder = 'heatmaps-fixed'
output_file = 'heatmap_video-fixed-range.avi'
fps = 4

def save_heatmaps_as_images(heatmaps, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for frame_number, heatmap in enumerate(heatmaps):
        
        if frame_number == 0:
            np_matrix = np.array(frames)
            max_val = np_matrix.max()
            min_val = np_matrix.min()
        
        plt.figure()
        sns.heatmap(heatmap, vmin = min_val, vmax = max_val, xticklabels = False, yticklabels = False)
        plt.savefig(f"{folder}/heatmap_{frame_number}.png")
        plt.close()

save_heatmaps_as_images(frames, folder)
create_video_from_images(folder, output_file, fps)

In [ ]:
np.array(frames).max()
np.array(frames).min()

row
column

In [ ]:
def interpolate_val(array_in):
    row, column = array_in.shape
    curr_row = 0
    while curr_row < row:
        curr_column = 0
        while curr_column < column:
            if array_in[curr_row][curr_column] == 0:
                
                if ((curr_row > 0) & (curr_column > 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/4)
                
                elif ((curr_row == 0) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == 0) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1])/2)
                
                elif (curr_row == (row-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/3)
                
                elif (curr_column == (column-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
                
                elif (curr_column == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)

                elif (curr_row == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
            print(curr_row, curr_column)
            curr_column+=1
            if curr_column > (column - 1):
                print("hello")
                break
        curr_row+=1
        if curr_row > (row - 1):
            break
    return array_in
interpolate_val(values_in_array)

In [ ]:
#This is final Code for Reading Values and Generating Image and Video files

In [ ]:
import serial as sr
import pandas as pd
import struct
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import cv2
import os

def interpolate_val(array_in):
    row, column = array_in.shape
    curr_row = 0
    while curr_row < row:
        curr_column = 0
        while curr_column < column:
            if array_in[curr_row][curr_column] == 0:
                
                if ((curr_row == 0) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == 0) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1])/2)

                elif ((curr_row > 0) & (curr_column > 0) & (curr_row != (row-1)) & (curr_column != (column-1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/4)
                
                
                elif (curr_row == (row-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/3)
                
                elif (curr_column == (column-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
                
                elif (curr_column == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)

                elif (curr_row == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
            # print(curr_row, curr_column)
            curr_column+=1
            if curr_column > (column - 1):
                
                break
        curr_row+=1
        if curr_row > (row - 1):
            break
    return array_in

def save_heatmaps_as_images(heatmaps, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for frame_number, heatmap in enumerate(heatmaps):
        
        if frame_number == 0:
            np_matrix = np.array(frames)
            max_val = np_matrix.max()
            min_val = np_matrix.min()
        
        plt.figure()
        # sns.heatmap(heatmap, vmax=max_val, vmin = min_val ,xticklabels = True, yticklabels = True)
        sns.heatmap(heatmap, cmap ="coolwarm", robust = True ,xticklabels = True, yticklabels = True)
        #sns.heatmap(heatmap, robust = True ,xticklabels = True, yticklabels = True)
        
        plt.savefig(f"{folder}/heatmap_{frame_number}.png")
        plt.close()

def generate_heatmap(data):
    # Normalize the data to the range [0, 255]
    normalized_data = cv2.normalize(data, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the normalized data to uint8
    normalized_data = normalized_data.astype(np.uint8)
    # Apply a colormap (e.g., COLORMAP_JET) to the normalized data
    heatmap = cv2.applyColorMap(normalized_data, cv2.COLORMAP_JET)
    return heatmap

#ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)

def read_until_end(serial_port, end_marker="end"):
    received_data = bytearray()
    end_marker_bytes = end_marker.encode()

    while True:
        if serial_port.in_waiting > 0:
            data = serial_port.read(serial_port.in_waiting)
            received_data.extend(data)

            if end_marker_bytes in received_data:
                #print("hello")
                break

    # Remove the end marker from the received data
    received_data = received_data.split(end_marker_bytes)[0]
    return received_data

# Create video from images
def create_video_from_images(folder, output_file, fps):
    images = [img for img in os.listdir(folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0])) # Sort images by frame number
    #print(int(x.split('_')[1].split('.')[0]))
    frame = cv2.imread(os.path.join(folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    
    for image in images:
        video.write(cv2.imread(os.path.join(folder, image)))
    
    cv2.destroyAllWindows()
    video.release()


rows = 24
columns = 32

#print(values)

vid_size = 100
current_frame = 0
folder = 'heatmaps'
output_file = 'heatmap_video.avi'
fps = 8
frames = []

while current_frame<vid_size:
    ser = sr.Serial(port="/dev/ttyACM3", baudrate = 921600)
    ser.flush()
    ser.write("Ready\r".encode('UTF-8'))
    
    value_from_sensor = read_until_end(ser)
    #print(value)
    values_in_array = np.zeros(shape=(rows, columns), dtype='float32')
    if len(value_from_sensor) == 3072:
        i = 0
        
        r = 0
        while r<24:
            c = 0
            k = 0
            while k<32:
                x = struct.unpack('<f', value_from_sensor[i:i+4])[0]
                # if x != 0:
                #     values_in_array[r,c] = x
                #     c+=1
                values_in_array[r,c] = x
                c+=1
                k+=1
                i+=4
            r+=1
        values_in_array = interpolate_val(values_in_array)
        print(current_frame)
        #print("     ")
        heatmap = generate_heatmap(values_in_array)
        cv2.imshow('Heatmap', heatmap)
        
        frames.append(values_in_array)    
        current_frame+=1
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

        # timer_val = bytearray()
        # timer_val.extend(ser.read(2))
        # data = struct.unpack('<H', value_from_sensor[0:2])[0]
        # print(data)

print("done Capturing")
save_heatmaps_as_images(frames, folder)
create_video_from_images(folder, output_file, fps)

#    return_val = ser.readline()
    

In [ ]:
#This Code Generates Live VIew

In [8]:
import serial as sr
import pandas as pd
import struct
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import cv2
import os

def serial_port_search():
    import serial.tools.list_ports
    plist = serial.tools.list_ports.comports()
    for i in enumerate(plist):
        # print(str(i[1]).split(' ')[2])
        strPort = str(i[1])
        if 'STLINK-V3' in strPort:
            port = strPort.split(' ')[0]
            # print(port)
            return port
        else:
            return 'None'

def interpolate_val(array_in):
    row, column = array_in.shape
    curr_row = 0
    while curr_row < row:
        curr_column = 0
        while curr_column < column:
            if array_in[curr_row][curr_column] == 0:
                
                if ((curr_row == 0) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == 0) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1])/2)

                elif ((curr_row > 0) & (curr_column > 0) & (curr_row != (row-1)) & (curr_column != (column-1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/4)
                
                
                elif (curr_row == (row-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/3)
                
                elif (curr_column == (column-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
                
                elif (curr_column == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)

                elif (curr_row == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
            # print(curr_row, curr_column)
            curr_column+=1
            if curr_column > (column - 1):
                
                break
        curr_row+=1
        if curr_row > (row - 1):
            break
    return array_in

def save_heatmaps_as_images(heatmaps, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for frame_number, heatmap in enumerate(heatmaps):
        
        if frame_number == 0:
            np_matrix = np.array(frames)
            max_val = np_matrix.max()
            min_val = np_matrix.min()
        
        plt.figure()
        # sns.heatmap(heatmap, vmax=max_val, vmin = min_val ,xticklabels = True, yticklabels = True)
        sns.heatmap(heatmap, cmap ="coolwarm", robust = True ,xticklabels = True, yticklabels = True)
        #sns.heatmap(heatmap, robust = True ,xticklabels = True, yticklabels = True)
        
        plt.savefig(f"{folder}/heatmap_{frame_number}.png")
        plt.close()

def generate_heatmap(data):
    # Normalize the data to the range [0, 255]
    # data = cv2.normalize(data, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the normalized data to uint8
    # data = data.astype(np.uint8)
    # normalized_data = (data[:][:]/10)*100
    
    # Apply a colormap (e.g., COLORMAP_JET) to the normalized data
    heatmap = cv2.applyColorMap(data, cv2.COLORMAP_MAGMA)
    return heatmap

#ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)

def read_until_end(serial_port, end_marker="end"):
    received_data = bytearray()
    end_marker_bytes = end_marker.encode()

    while True:
        if serial_port.in_waiting > 0:
            data = serial_port.read(serial_port.in_waiting)
            received_data.extend(data)

            if end_marker_bytes in received_data:
                #print("hello")
                break

    # Remove the end marker from the received data
    received_data = received_data.split(end_marker_bytes)[0]
    return received_data

# Create video from images
def create_video_from_images(folder, output_file, fps):
    images = [img for img in os.listdir(folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0])) # Sort images by frame number
    #print(int(x.split('_')[1].split('.')[0]))
    frame = cv2.imread(os.path.join(folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    
    for image in images:
        video.write(cv2.imread(os.path.join(folder, image)))
    
    cv2.destroyAllWindows()
    video.release()

def normalize_array(not_nor_data, max, min):
    range = max - min
    not_nor_data[not_nor_data > max] = max
    not_nor_data[not_nor_data < min] = min
    not_nor_data = ((not_nor_data - min) / range) * 255

    not_nor_data = not_nor_data.astype(np.uint8)
    # print(not_nor_data)
    return not_nor_data


def put_temp_val(input_array, image_data, img_x, img_y):
    x_size = input_array.shape[1]
    #print(x_size)
    y_size = input_array.shape[0]
    #print(y_size)
    y = 0
    while y<y_size:
        x = 0
        while x<x_size:
            cv2.putText(image_data, "{:.2f}".format(input_array[y][x]), ((int((img_x/x_size)*x)), (int((img_y/y_size)*y)+15)), cv2.FONT_HERSHEY_SIMPLEX,.35, (255, 255, 255), 1)
            x+=1
        y+=1

def put_human_val(input_array, image_data, img_x, img_y):
    x_size = input_array.shape[1]
    y_size = input_array.shape[0]
    y = 0
    count = 0
    while y<y_size:
        x = 0
        while x<x_size:
            if(input_array[y][x] > 2.2) & (input_array[y][x] < 7):
                count += 1
                cv2.rectangle(image_data, ((int((img_x/x_size)*x)), (int((img_y/y_size)*y)+40)), ((int((img_x/x_size)*x)+40), (int((img_y/y_size)*y))), (0,255,0),-1)
            x+=1
        y+=1
    print('Human Pixel Count')
    print(count)

rows = 24
columns = 32

#print(values)

vid_size = 10000
current_frame = 0
folder = 'heatmaps-dataset-3-sleeping-bare'
output_file = 'heatmap_video.avi'
fps = 8
frames = []

# while current_frame<vid_size:
if not os.path.exists(folder):
    os.makedirs(folder)

while 1:
    ser = sr.Serial(port='/dev/ttyUSB5', baudrate = 921600)
    vid = cv2.VideoWriter("heatmap-cv2-19.avi",cv2.VideoWriter_fourcc(*'DIVX'),8,(1280,960))
    
    while 1:
        ser.flush()
        ser.write("ready".encode('UTF-8'))
        
        value_from_sensor = read_until_end(ser)
        
        #print(value)
        values_in_array = np.zeros(shape=(rows, columns), dtype='float32')
        
        if len(value_from_sensor) == 3072:
            i = 0
            
            r = 0
            while r<24:
                c = 0
                k = 0
                while k<32:
                    x = struct.unpack('<f', value_from_sensor[i:i+4])[0]
                    # if x != 0:
                    #     values_in_array[r,c] = x
                    #     c+=1
                    values_in_array[r,c] = x
                    c+=1
                    k+=1
                    i+=4
                r+=1
            #values_in_array = interpolate_val(values_in_array)
            #print(current_frame)
            #print("     ")

            #values_in_array = np.log(values_in_array)
            raw_data = values_in_array
            normal_data = normalize_array(values_in_array, 45, 0)
            
            heatmap = generate_heatmap(normal_data)
            
            img = cv2.resize(heatmap, (0, 0), cv2.INTER_CUBIC,40,40)
            put_human_val(values_in_array, img, 1280, 960)
            put_temp_val(raw_data, img, 1280, 960)
            
            vid.write(img)
            cv2.imshow('Heatmap', img)
            cv2.imwrite(f"{folder}/img_{current_frame}.png", img)
            
            # img = 0
            #clusterdata = ser.read(ser.in_waiting)
            #print(clusterdata)
            # cv2.namedWindow('Heatmap', cv2.WINDOW_NORMAL)
            # frames.append(values_in_array)    
            current_frame+=1
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
    
    
            # timer_val = bytearray()
            # timer_val.extend(ser.read(2))
            # data = struct.unpack('<H', value_from_sensor[0:2])[0]
            # print(data)
    print("done Capturing")
    # save_heatmaps_as_images(frames, folder)
    # create_video_from_images(folder, output_file, fps)
    cv2.destroyAllWindows()
    vid.release()
    print("done release")
    break

#    return_val = ser.readline()



Human Pixel Count
0


libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
Human Pixel Count
0
done Capturing
done release


In [1]:
cv2.__version__

NameError: name 'cv2' is not defined

In [ ]:
import cv2
cv2.__version__

In [ ]:
import serial.tools.list_ports
serial.tools.list_ports.comports()


In [ ]:
for i in values_in_array:
    print(i)

In [ ]:

def serial_port_search():
    import serial.tools.list_ports
    plist = serial.tools.list_ports.comports()
    for i in enumerate(plist):
        # print(str(i[1]).split(' ')[2])
        strPort = str(i[1])
        if 'STLINK-V3' in strPort:
            port = strPort.split(' ')[0]
            print(port)
            return port
        else:
            return 'None'
        


In [ ]:
def put_human_val(input_array, image_data, img_x, img_y):
    x_size = input_array.shape[0]
    y_size = input_array.shape[1]
    x = 0
    while x<x_size:
        y = 0
        while y<y_size:
            if(input_array[y][x] > 32.5 & input_array[y][x] < 35):
                cv2.rectangle(image_data, ((int((img_x/x_size)*x)), (int((img_y/y_size)*y)+40)), ((int((img_x/x_size)*x)+40), (int((img_y/y_size)*y))), (0,0,0),-1)
            y+=1
        x+=1

    

In [ ]:
put_temp_val(values_in_array, heatmap, 640,480)

In [ ]:
normal_data = normalize_array(values_in_array, 40, 15)
heatmap = generate_heatmap(normal_data)

img = cv2.resize(heatmap, (0, 0), cv2.INTER_NEAREST,40,40)
put_temp_val(values_in_array, img, 960, 1280)
cv2.imshow('Heatmap', img)

In [ ]:
print(values_in_array)
values_in_array[0][3]

In [64]:
# Get list of all files in the folder
folder_path = "/home/samirhassan/Downloads/images"
files = os.listdir(folder_path)

# Filter out .jpg and .xml files
jpg_files = sorted([f for f in files if f.endswith(".jpg")])
xml_files = sorted([f for f in files if f.endswith(".xml")])

# Ensure that the number of jpg and xml files match
if len(jpg_files) != len(xml_files):
    print("The number of .jpg and .xml files does not match!")
else:
    # Rename files in pairs
    for index, (jpg_file, xml_file) in enumerate(zip(jpg_files, xml_files), start=1):
        new_jpg_name = f"img_{index}.jpg"
        new_xml_name = f"img_{index}.xml"

        # Full paths for renaming
        old_jpg_path = os.path.join(folder_path, jpg_file)
        new_jpg_path = os.path.join(folder_path, new_jpg_name)
        
        old_xml_path = os.path.join(folder_path, xml_file)
        new_xml_path = os.path.join(folder_path, new_xml_name)

        # Rename the files
        os.rename(old_jpg_path, new_jpg_path)
        os.rename(old_xml_path, new_xml_path)

        print(f"Renamed {jpg_file} to {new_jpg_name}")
        print(f"Renamed {xml_file} to {new_xml_name}")

print("Renaming completed.")

Renamed img1.jpg to img_1.jpg
Renamed img1.xml to img_1.xml
Renamed img10.jpg to img_2.jpg
Renamed img10.xml to img_2.xml
Renamed img11.jpg to img_3.jpg
Renamed img11.xml to img_3.xml
Renamed img12.jpg to img_4.jpg
Renamed img12.xml to img_4.xml
Renamed img13.jpg to img_5.jpg
Renamed img13.xml to img_5.xml
Renamed img14.jpg to img_6.jpg
Renamed img14.xml to img_6.xml
Renamed img15.jpg to img_7.jpg
Renamed img15.xml to img_7.xml
Renamed img16.jpg to img_8.jpg
Renamed img16.xml to img_8.xml
Renamed img17.jpg to img_9.jpg
Renamed img17.xml to img_9.xml
Renamed img18.jpg to img_10.jpg
Renamed img18.xml to img_10.xml
Renamed img19.jpg to img_11.jpg
Renamed img19.xml to img_11.xml
Renamed img2.jpg to img_12.jpg
Renamed img2.xml to img_12.xml
Renamed img20.jpg to img_13.jpg
Renamed img20.xml to img_13.xml
Renamed img21.jpg to img_14.jpg
Renamed img21.xml to img_14.xml
Renamed img22.jpg to img_15.jpg
Renamed img22.xml to img_15.xml
Renamed img23.jpg to img_16.jpg
Renamed img23.xml to img_16.xm

In [156]:
# Get list of all files in the folder
folder_path = "/home/samirhassan/Downloads/final_dataset"
files = os.listdir(folder_path)

# Filter out .jpg and .xml files
jpg_files = sorted([f for f in files if f.endswith(".png")])

# Ensure that the number of jpg and xml files match
#if len(jpg_files) != len(xml_files):
#    print("The number of .jpg and .xml files does not match!")
print(files)
# Rename files in pairs
for index, jpg_file in enumerate((jpg_files), start=1):
    new_jpg_name = f"img_{index}.png"

    # Full paths for renaming
    old_jpg_path = os.path.join(folder_path, jpg_file)
    new_jpg_path = os.path.join(folder_path, new_jpg_name)


    # Rename the files
    os.rename(old_jpg_path, new_jpg_path)
    print(index)
    print(f"Renamed {jpg_file} to {new_jpg_name}")

print("Renaming completed.")

['imge_18.png', 'imge_4.png', 'imgd_17.png', 'imga_37.png', 'imge_15.png', 'imge_19.png', 'imge_14.png', 'imga_21.png', 'imga_6.png', 'imge_5.png', 'imga_34.png', 'imga_32.png', 'imga_30.png', 'imgd_20.png', 'imgd_5.png', 'imgb_13.png', 'imge_11.png', 'imga_23.png', 'imgc_9.png', 'imga_35.png', 'imgd_1.png', 'imgb_16.png', 'imgd_21.png', 'imga_18.png', 'imga_19.png', 'imga_28.png', 'imgc_7.png', 'imga_17.png', 'imgd_9.png', 'imga_4.png', 'imgd_3.png', 'imgd_2.png', 'imge_13.png', 'imga_13.png', 'imga_1.png', 'imgb_5.png', 'imgd_15.png', 'imgb_3.png', 'imgc_2.png', 'imga_12.png', 'imga_7.png', 'imgd_7.png', 'imgb_17.png', 'imge_7.png', 'imga_25.png', 'imge_1.png', 'imgc_3.png', 'imge_3.png', 'imge_6.png', 'imga_33.png', 'imgd_14.png', 'imgc_11.png', 'imgc_8.png', 'imgd_8.png', 'imge_17.png', 'imgd_22.png', 'imge_8.png', 'imga_16.png', 'imgc_13.png', 'imgd_13.png', 'imga_24.png', 'imge_9.png', 'imga_20.png', 'imgb_2.png', 'imgc_6.png', 'imgd_10.png', 'imgb_6.png', 'imgc_15.png', 'imgd_19

In [7]:
#this code is lastest with calibration

In [ ]:
import serial as sr
import pandas as pd
import struct
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import cv2
import os

def serial_port_search():
    import serial.tools.list_ports
    plist = serial.tools.list_ports.comports()
    for i in enumerate(plist):
        # print(str(i[1]).split(' ')[2])
        strPort = str(i[1])
        if 'STLINK-V3' in strPort:
            port = strPort.split(' ')[0]
            # print(port)
            return port
        else:
            return 'None'

def interpolate_val(array_in):
    row, column = array_in.shape
    curr_row = 0
    while curr_row < row:
        curr_column = 0
        while curr_column < column:
            if array_in[curr_row][curr_column] == 0:
                
                if ((curr_row == 0) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == 0) & (curr_column == (column - 1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1])/2)
                
                elif ((curr_row == (row-1)) & (curr_column == 0)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1])/2)

                elif ((curr_row > 0) & (curr_column > 0) & (curr_row != (row-1)) & (curr_column != (column-1))):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/4)
                
                
                elif (curr_row == (row-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row-1][curr_column] + array_in[curr_row][curr_column+1] + array_in[curr_row][curr_column-1])/3)
                
                elif (curr_column == (column-1)):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
                
                elif (curr_column == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)

                elif (curr_row == 0):
                    array_in[curr_row][curr_column] = ((array_in[curr_row+1][curr_column] + array_in[curr_row][curr_column-1] + array_in[curr_row-1][curr_column])/3)
            # print(curr_row, curr_column)
            curr_column+=1
            if curr_column > (column - 1):
                
                break
        curr_row+=1
        if curr_row > (row - 1):
            break
    return array_in

def save_heatmaps_as_images(heatmaps, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for frame_number, heatmap in enumerate(heatmaps):
        
        if frame_number == 0:
            np_matrix = np.array(frames)
            max_val = np_matrix.max()
            min_val = np_matrix.min()
        
        plt.figure()
        # sns.heatmap(heatmap, vmax=max_val, vmin = min_val ,xticklabels = True, yticklabels = True)
        sns.heatmap(heatmap, cmap ="coolwarm", robust = True ,xticklabels = True, yticklabels = True)
        #sns.heatmap(heatmap, robust = True ,xticklabels = True, yticklabels = True)
        
        plt.savefig(f"{folder}/heatmap_{frame_number}.png")
        plt.close()

def generate_heatmap(data):
    # Normalize the data to the range [0, 255]
    # data = cv2.normalize(data, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the normalized data to uint8
    # data = data.astype(np.uint8)
    # normalized_data = (data[:][:]/10)*100
    
    # Apply a colormap (e.g., COLORMAP_JET) to the normalized data
    heatmap = cv2.applyColorMap(data, cv2.COLORMAP_MAGMA)
    return heatmap

#ser = sr.Serial(port="/dev/ttyACM1", baudrate = 115200)

def read_until_end(serial_port, end_marker="end"):
    received_data = bytearray()
    end_marker_bytes = end_marker.encode()

    while True:
        if serial_port.in_waiting > 0:
            data = serial_port.read(serial_port.in_waiting)
            received_data.extend(data)

            if end_marker_bytes in received_data:
                #print("hello")
                break

    # Remove the end marker from the received data
    received_data = received_data.split(end_marker_bytes)[0]
    return received_data

# Create video from images
def create_video_from_images(folder, output_file, fps):
    images = [img for img in os.listdir(folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0])) # Sort images by frame number
    #print(int(x.split('_')[1].split('.')[0]))
    frame = cv2.imread(os.path.join(folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    
    for image in images:
        video.write(cv2.imread(os.path.join(folder, image)))
    
    cv2.destroyAllWindows()
    video.release()

def normalize_array(not_nor_data, max, min):
    range = max - min
    not_nor_data[not_nor_data > max] = max
    not_nor_data[not_nor_data < min] = min
    not_nor_data = ((not_nor_data - min) / range) * 255

    not_nor_data = not_nor_data.astype(np.uint8)
    # print(not_nor_data)
    return not_nor_data


def put_temp_val(input_array, image_data, img_x, img_y):
    x_size = input_array.shape[1]
    #print(x_size)
    y_size = input_array.shape[0]
    #print(y_size)
    y = 0
    while y<y_size:
        x = 0
        while x<x_size:
            cv2.putText(image_data, "{:.2f}".format(input_array[y][x]), ((int((img_x/x_size)*x)), (int((img_y/y_size)*y)+15)), cv2.FONT_HERSHEY_SIMPLEX,.35, (255, 255, 255), 1)
            x+=1
        y+=1

def put_human_val(input_array, image_data, img_x, img_y):
    x_size = input_array.shape[1]
    y_size = input_array.shape[0]
    y = 0
    count = 0
    while y<y_size:
        x = 0
        while x<x_size:
            if(input_array[y][x] > 32.6) & (input_array[y][x] < 36.5):
                count += 1
                cv2.rectangle(image_data, ((int((img_x/x_size)*x)), (int((img_y/y_size)*y)+40)), ((int((img_x/x_size)*x)+40), (int((img_y/y_size)*y))), (0,0,0),-1)
            x+=1
        y+=1
    print('Human Pixel Count')
    print(count)

rows = 24
columns = 32

#print(values)

vid_size = 10000
current_frame = 0
folder = 'heatmaps-dataset-3-sleeping-bare'
output_file = 'heatmap_video.avi'
fps = 8
frames = []

# while current_frame<vid_size:
if not os.path.exists(folder):
    os.makedirs(folder)

while 1:
    ser = sr.Serial(port='/dev/ttyUSB0', baudrate = 921600)
    vid = cv2.VideoWriter("heatmap-cv2-19.avi",cv2.VideoWriter_fourcc(*'DIVX'),8,(1280,960))
    calib_count = 0
    frame_count = 0
    calib_array = np.zeros(shape=(rows, columns), dtype='float32')
    values_in_array = np.zeros(shape=(rows, columns), dtype='float32')
    while 1:
        ser.flush()
        #stm32
        #ser.write("Ready\r".encode('UTF-8'))
        #for esp32
        ser.write("ready".encode('UTF-8'))
        
        value_from_sensor = read_until_end(ser)
        #print(value)
        
        
        if len(value_from_sensor) == 3072:
            if calib_count < 5:
                i = 0
                r = 0
                while r<24:
                    c = 0
                    k = 0
                    while k<32:
                        x = struct.unpack('<f', value_from_sensor[i:i+4])[0]
                        
                        calib_array[r,c] = calib_array[r,c] + x
                        c+=1
                        k+=1
                        i+=4
                    r+=1
                calib_count += 1
                print(calib_count)
                if calib_count == 5:
                    calib_array = calib_array/5;
                    #print(calib_array)
            else:
                if frame_count < 2:
                    i = 0
                    r = 0
                    while r<24:
                        c = 0
                        k = 0
                        while k<32:
                            x = struct.unpack('<f', value_from_sensor[i:i+4])[0]
                            # if x != 0:
                            #     values_in_array[r,c] = x
                            #     c+=1
                            values_in_array[r,c] = values_in_array[r,c] + x
                            c+=1
                            k+=1
                            i+=4
                        r+=1
                    frame_count+=1
                    
                    if frame_count == 2:
                        values_in_array = values_in_array[:,:]/2;
                        
                        #values_in_array = interpolate_val(values_in_array)
                        values_in_array = values_in_array - calib_array
                        
                        values_in_array[values_in_array < 0]  = 0

                        #use with log function
                        values_in_array = values_in_array * 10;
                        
                        logvalues_in_array = np.log(values_in_array)
                        
                        #use this is for usage without log function
                        #normal_data = normalize_array(values_in_array, np.max(values_in_array), np.min(values_in_array))
                        
                        #use this with log
                        normal_data = normalize_array(logvalues_in_array, 10, 0)
                        
                        heatmap = generate_heatmap(normal_data)
                        
                        img = cv2.resize(heatmap, (0, 0), cv2.INTER_CUBIC,40,40)
                        #put_human_val(values_in_array, img, 1280, 960)
                        put_temp_val(logvalues_in_array, img, 1280, 960)
                        
                        vid.write(img)
                        cv2.imshow('Heatmap', img)
                        cv2.imwrite(f"{folder}/img_{current_frame}.png", img)
                        frame_count = 0
                        
                        # img = 0
                        # cv2.namedWindow('Heatmap', cv2.WINDOW_NORMAL)
                        # frames.append(values_in_array)   
                        if cv2.waitKey(100) & 0xFF == ord('q'):
                            break
                        values_in_array = np.zeros(shape=(rows, columns), dtype='float32')

                            
        
        
                # timer_val = bytearray()
                # timer_val.extend(ser.read(2))
                # data = struct.unpack('<H', value_from_sensor[0:2])[0]
                # print(data)q
        

    print("done Capturing")
    # save_heatmaps_as_images(frames, folder)
    # create_video_from_images(folder, output_file, fps)
    cv2.destroyAllWindows()
    vid.release()
    print("done release")
    break

#    return_val = ser.readline()



1
2
3
4
5


/tmp/ipykernel_21278/2314363070.py:257: RuntimeWarning: divide by zero encountered in log
  logvalues_in_array = np.log(values_in_array)
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [3]:
values_in_array
img = cv2.resize(calib_array, (0, 0), cv2.INTER_BITS2,40,40)
cv2.imshow('Heatmap', img)

NameError: name 'values_in_array' is not defined